This script calculates the distance of the safecast data points from every unique mext location

In [1]:
from math import cos, sqrt, sin
import pandas as pd
import numpy as np

mext_rad = [(0.5960859297184675,2.2939612326941052), #values copied from mext_loc_rad.csv
                (0.6191568611419274,2.321510923324278),
                (0.6225977254023566,2.418148826622823),
                (0.6387957422176805,2.4424137552408647),
                (0.4592785406469872,2.2321936988534685),
                (0.6351592440013953,2.4508883061436784),
                (0.6053383564156025,2.359161549920735),
                (0.6000441968356506,2.3120085654782576),
                (0.5555945354182943,2.2936360778544587),
                (0.6166490673529068,2.432717473861655),
                (0.6036915509998833,2.336440120679987),
                (0.6680723314229464,2.459266741764585),
                (0.580731151277952,2.273686475811973),
                (0.626025429880226,2.436701554398475),
                (0.6052121167508057,2.370479678507361),
                (0.617980369599743,2.3883827249222502),
                (0.6605295594478952,2.4250137999828625),
                (0.6194755233567565,2.336734069032608),
                (0.5857800048315361,2.3305786767332592),
                (0.5512641815578787,2.2787808999123267),
                (0.6200427553636546,2.4446632926604677),
                (0.6106095998224758,2.3718827137864538),
                (0.6405484367591183,2.392835740522496),
                (0.6588680932664591,2.451653336314705),
                (0.5995244901442841,2.3400475766175193),
                (0.623195308684947,2.438188784360684),
                (0.6394100981143827,2.4116726951668355),
                (0.6108978758550277,2.4152896806020836),
                (0.5905896784644271,2.3170033312783227),
                (0.637518841883629,2.385965932600429),
                (0.6353792951134868,2.4276827749283045),
                (0.6052801845916335,2.365536853705835),
                (0.6925112519141194,2.463238919155906),
                (0.6296090969800533,2.3782152046450022),
                (0.6107245646603046,2.382112629584461),
                (0.6932350748615067,2.445666856980364),
                (0.5971498649771907,2.359025798211515),
                (0.5849017202453477,2.2776580248847638),
                (0.594633676154468,2.348529580076239),
                (0.6143995299333038,2.389792078293236),
                (0.5787196593150287,2.2970908523890268),
                (0.7126179335892847,2.4572829830834753),
                (0.574896917014263,2.2685498671035993),
                (0.6096774369222782,2.3694153592762017),
                (0.6675641962645208,2.4493029537709217),
                (0.6675641962645208,2.4493029537709217),
                (0.7519392242523358,2.4667271516522344),
                (0.5787196593150287,2.2803260049925322)]

def equirect_approx(lat1,lng1,lat2,lng2):
    """
    values must be in radians"""
    x = (lng2-lng1)*cos((lat1+lat2)/2.0)
    y = lat2-lat1
    return sqrt(x*x + y*y) * 6371.0 #earth radius in km

def haversine_distance(lat1,lng1,lat2,lng2):
    R = 6731.0 #radius of Earth in km
    dlon = lng2 - lng1
    dlat = lat2 - lat1
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2 
    c = 2 * np.arctan2(sqrt(a), sqrt(1-a)) 
    return 6731.0 * c

def dist_from_all_mext(latd,lngd):
    """
    inputs are in degrees"""
    
    lat,lng = map(radians, (latd,lngd))
#     return [round(equirect_approx(m[0],m[1],lat,lng),5) for m in mext_rad] #rounded to save disk space and time, equirect approximation
    return [round(haversine_distance(m[0],m[1],lat,lng),5) for m in mext_rad] #haversine formula
#     return [equirect_approx(m[0],m[1],lat,lng) for m in mext_rad] #unrounded

def min_distance(mext_distances):
    """
    returns the minimum distance from any point, lat of that point, long of that point
    """
    ind = np.argmin(mext_distances)
#     loc = mext_rad[ind]
#     return [mext_distances[ind],loc[0],loc[1]]
    return [mext_distances[ind],ind] #saves the mext location as a number corresponding to the index in mext_rad list object

In [6]:
#goes through the safecast data (time filtered) and calculates the closest point of all the mext points
#saves two separate files
#1. all the safecast data plus closest point information
#2. only closest point information

import timeit
import csv
from math import radians
t = timeit.default_timer()
with open('safecast_data_and_dists.csv','w') as dwf:
    dwriter = csv.writer(dwf)
    with open('safecast_dists.csv','w') as wf:
        writer = csv.writer(wf)
        with open('safecast_data_datefiltered.csv','r') as sf:
            i=0
            #write header: distance from point at:, lat, long
            dwriter.writerow(next(sf).strip().split(',')+['distance','mlat','mlong'])
            writer.writerow(['distance','mlat','mlong'])
            for line in sf:
                i+=1
                l=line.split(',')
                l[5] = l[5].strip()
    #             print(dist_from_all_mext(*map(float,(l[1:3])))[0])
                try:
    #                 writer.writerow(dist_from_all_mext(*map(float,(l[1:3])))) #only write distances
                    dist_lat_long = min_distance(dist_from_all_mext(*map(float,(l[1:3]))))
                    dwriter.writerow(l+dist_lat_long)
                    writer.writerow(dist_lat_long)
                except ValueError:
                    writer.writerow(['nan']*3)
                    dwriter.writerow(l+['nan']*3)
#                 if i%1000000==0:
                if i%100000==0:
                    print(i,timeit.default_timer()-t)
#                     break


100000 36.050405745001626
200000 76.57824690399866
300000 111.69045346900384
400000 150.1384815679994
500000 189.27876615199784
600000 226.50824566800293
700000 267.87348839199694
800000 304.5248522559996
900000 340.20368423999753
1000000 374.6853878050024
1100000 416.51570078400255
1200000 454.60368899699824
1300000 489.8141300350035
1400000 525.9596874010022
1500000 562.0809432469978
1600000 598.5152850480008
1700000 635.3528140760027
1800000 671.3238133170016
1900000 706.4694633670006
2000000 741.5637312269973
2100000 777.1593653700038
